# Смыслов Дмитрий Олегович, ИУ5-25М
# Рубежный контроль №2
## Тема: Методы обработки текстов
## Решение задачи классификации текстов

Необходимо решить задачу классификации текстов на основе любого выбранного Вами датасета (кроме примера, который рассматривался в лекции). Классификация может быть бинарной или многоклассовой. Целевой признак из выбранного Вами датасета может иметь любой физический смысл, примером является задача анализа тональности текста.

Необходимо сформировать два варианта векторизации признаков - на основе CountVectorizer и на основе TfidfVectorizer.

В качестве классификаторов необходимо использовать два классификатора по варианту для Вашей группы:
ИУ5-25М, ИУ5И-25М, ИУ5-25МВ: SVC и LogisticRegression

Датасет "habr-posts" - https://www.kaggle.com/datasets/leadness/habr-posts

Содержит информацию о постах на хабре и их тегах (связаны отношением многие ко многим)

Будем решать задачу классификации текста на основе их тегов (тегирование текста)

Каждому тексту будем ставить в соответствие несколько тегов

Для этого будем обучать модели по принципу Один против всех

Из-за объема датасета (~200 тыс. уникальных тегов и ~230 тыс. записей, сам датасет весит ~5 ГБ) количество записей и тегов было ограничено

In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score, hamming_loss, precision_score, recall_score
import warnings
warnings.filterwarnings('ignore')

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Предобработка текста - фильтрация стоп слов, пунктуации
def preprocess_text(text):
    if not isinstance(text, str):  # Handle non-string (e.g., NaN)
        return ''
    tokens = word_tokenize(text.lower())
    tokens = [t for t in tokens if t not in stop_words and t not in string.punctuation]
    return ' '.join(tokens)

In [6]:
# Загрузка датасета
posts_df = pd.read_csv('/kaggle/input/habr-posts/posts.csv')
tags_df = pd.read_csv('/kaggle/input/habr-posts/tags.csv')

print("Posts DataFrame Info:")
print(posts_df.info())

print("\nTags DataFrame Info:")
print(tags_df.info())

Posts DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232127 entries, 0 to 232126
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Unnamed: 0       232127 non-null  int64 
 1   post_id          232127 non-null  int64 
 2   title            232127 non-null  object
 3   text             231467 non-null  object
 4   date             232127 non-null  object
 5   views_count      232127 non-null  int64 
 6   comments_count   232127 non-null  int64 
 7   bookmarks_count  232127 non-null  int64 
 8   rating           232127 non-null  object
 9   author_nickname  232120 non-null  object
dtypes: int64(5), object(5)
memory usage: 17.7+ MB
None

Tags DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054820 entries, 0 to 1054819
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Unnamed: 0  1054820 non-null  int64 

In [7]:
# Оставляем только текст и post_id
posts_df.drop(columns=[
    'title',
    'date',
    'views_count',
    'comments_count',
    'bookmarks_count',
    'rating',
    'author_nickname'
], inplace=True)

In [8]:
posts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232127 entries, 0 to 232126
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  232127 non-null  int64 
 1   post_id     232127 non-null  int64 
 2   text        231467 non-null  object
dtypes: int64(2), object(1)
memory usage: 5.3+ MB


In [9]:
posts_df.head(3)

,Unnamed: 0,post_id,text
0,0,365293,Разработчики LinkedIn объявили о появившейся в...
1,1,5005,Со своего мобильника вы заказываете фальшивый ...
2,2,125101,Jelastic — облачный хостинг для ранее разработ...


In [10]:
# tags_df = pd.read_csv('/kaggle/input/habr-posts/tags.csv')

In [11]:
# Количество уникальных тегов и их частота
# Всего уникальных тегов - 204.527
# Для предсказывания нескольких тегов для текстов
# воспользуемся OneVsRestClassifier и ограничим число тегов
print(tags_df['tag'].value_counts())

tag
javascript                        4473
android                           4011
google                            3726
linux                             3274
php                               3224
                                  ... 
джун                                 1
кризисный менеджмент                 1
Meteor живи                          1
Google DoubleClick Ad Exchange       1
lightswitch 2011                     1
Name: count, Length: 204527, dtype: int64


In [12]:
# Интересующие теги
fav_tags = ['rust', 'javascript', 'go', 'linux', 'bugs']

In [13]:
tags_df = tags_df[tags_df['tag'].isin(fav_tags)]
tags_df

,Unnamed: 0,post_id,tag
138,138,55045,linux
249,249,70058,linux
524,524,480398,javascript
564,564,485430,javascript
571,571,430378,javascript
...,...,...,...
1053877,1053877,115060,linux
1054062,1054062,124998,linux
1054240,1054240,120052,linux
1054315,1054315,485362,javascript


In [14]:
# Агрегация тегов в список
tags_df['tag'] = tags_df['tag'].astype(str)
tags_df = tags_df.groupby('post_id')['tag'].apply(list).reset_index()
tags_df.head()

,post_id,tag
0,2,[bugs]
1,274,[linux]
2,305,[linux]
3,307,[linux]
4,339,[linux]


In [15]:
# Распределение длин списков тегов
tags_df['tag'].apply(len).value_counts()

tag
1    8505
2      67
3       5
Name: count, dtype: int64

In [16]:
# Сливаем тексты с их тегами
df = posts_df.merge(tags_df, on='post_id', how='inner')

# Удаляем строки с пропусками
df = df.dropna(subset=['text', 'tag'])
df = df[df['text'].str.strip() != '']
df = df[df['tag'].str.strip() != '']

print(df.info())
df.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 8529 entries, 0 to 8576
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8529 non-null   int64 
 1   post_id     8529 non-null   int64 
 2   text        8529 non-null   object
 3   tag         8529 non-null   object
dtypes: int64(2), object(2)
memory usage: 333.2+ KB
None


,Unnamed: 0,post_id,text,tag
0,34,55045,Поставили задачу установить dhcp — сервер в не...,[linux]
1,50,2,"Кроме неработающих и отсутствующих страниц, о ...",[bugs]
2,69,485404,"\n\nДоброго времени суток, друзья!\n\nНа днях ...",[javascript]


In [17]:
# Ограничим число записей
num_records = min(10_000, df.shape[0])
df = df.sample(n=num_records, random_state=42)

# Предобработка текста
df['text'] = df['text'].apply(preprocess_text)
df = df[df['text'].str.strip() != '']

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8529 entries, 2517 to 7313
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8529 non-null   int64 
 1   post_id     8529 non-null   int64 
 2   text        8529 non-null   object
 3   tag         8529 non-null   object
dtypes: int64(2), object(2)
memory usage: 333.2+ KB


In [18]:
# Кодирование тегов
mlb = MultiLabelBinarizer(sparse_output=True)
y = mlb.fit_transform(df['tag'])

In [19]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], y, test_size=0.2, random_state=42
)

In [51]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [21]:
# CountVectorizer
count_vectorizer = CountVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

In [22]:
# Создание моделей
tfidf_lr = OneVsRestClassifier(LogisticRegression(max_iter=1000), n_jobs=-1)
tfidf_svc = OneVsRestClassifier(SVC(probability=True), n_jobs=-1)
count_lr = OneVsRestClassifier(LogisticRegression(max_iter=1000), n_jobs=-1)
count_svc = OneVsRestClassifier(SVC(probability=True), n_jobs=-1)

In [23]:
def train(model, X_train, X_test, y_train, y_test):
    # Обучение модели
    model.fit(X_train, y_train)
    
    # Предсказание на тестовой выборке
    y_pred = model.predict(X_test)
    
    # Оценка метрик
    f1 = f1_score(y_test, y_pred, average="micro")
    precision = precision_score(y_test, y_pred, average="micro")
    recall = recall_score(y_test, y_pred, average="micro")
    
    print(f"F1-score: {f1:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(classification_report(y_test, y_pred, target_names=mlb.classes_))

In [24]:
# TfidfVectorizer + LogisticRegression
train(tfidf_lr, X_train_tfidf, X_test_tfidf, y_train, y_test)

F1-score: 0.939
Precision: 0.973
Recall: 0.908
              precision    recall  f1-score   support

        bugs       0.88      0.23      0.37        30
          go       0.99      0.59      0.74       116
  javascript       0.98      0.97      0.97       884
       linux       0.97      0.94      0.95       640
        rust       1.00      0.58      0.73        52

   micro avg       0.97      0.91      0.94      1722
   macro avg       0.96      0.66      0.75      1722
weighted avg       0.97      0.91      0.93      1722
 samples avg       0.92      0.91      0.91      1722



In [25]:
# CountVectorizer + LogisticRegression
train(count_lr, X_train_count, X_test_count, y_train, y_test)

F1-score: 0.940
Precision: 0.953
Recall: 0.927
              precision    recall  f1-score   support

        bugs       0.85      0.37      0.51        30
          go       0.97      0.82      0.89       116
  javascript       0.97      0.95      0.96       884
       linux       0.94      0.95      0.94       640
        rust       0.89      0.79      0.84        52

   micro avg       0.95      0.93      0.94      1722
   macro avg       0.92      0.78      0.83      1722
weighted avg       0.95      0.93      0.94      1722
 samples avg       0.93      0.93      0.93      1722



In [26]:
# CountVectorizer + SVC
train(count_svc, X_train_count, X_test_count, y_train, y_test)

F1-score: 0.915
Precision: 0.973
Recall: 0.864
              precision    recall  f1-score   support

        bugs       0.57      0.13      0.22        30
          go       0.99      0.60      0.75       116
  javascript       0.98      0.88      0.93       884
       linux       0.96      0.94      0.95       640
        rust       0.97      0.67      0.80        52

   micro avg       0.97      0.86      0.91      1722
   macro avg       0.89      0.65      0.73      1722
weighted avg       0.97      0.86      0.91      1722
 samples avg       0.87      0.87      0.87      1722



In [61]:
# TfidfVectorizer + SVC
train(tfidf_svc, X_train_tfidf.toarray(), X_test_tfidf.toarray(), y_train, y_test)

F1-score: 0.950
Precision: 0.972
Recall: 0.929
              precision    recall  f1-score   support

        bugs       0.76      0.43      0.55        30
          go       0.99      0.76      0.86       116
  javascript       0.98      0.97      0.97       884
       linux       0.97      0.94      0.95       640
        rust       1.00      0.77      0.87        52

   micro avg       0.97      0.93      0.95      1722
   macro avg       0.94      0.77      0.84      1722
weighted avg       0.97      0.93      0.95      1722
 samples avg       0.94      0.93      0.93      1722



In [62]:
# Все модели
models = {
    'TfidfVectorizer + LogisticRegression': (tfidf_vectorizer, tfidf_lr),
    'TfidfVectorizer + SVC': (tfidf_vectorizer, tfidf_svc),
    'CountVectorizer + LogisticRegression': (count_vectorizer, count_lr),
    'CountVectorizer + SVC': (count_vectorizer, count_svc)
}

In [67]:
def extract_tags(text):
    for name, (vectorizer, model) in models.items():
        try:
            new_text_vectorized = vectorizer.transform([text])
            pred = model.predict(new_text_vectorized)
            predicted_labels = mlb.inverse_transform(pred)[0]
            print(f"{name}: {', '.join(predicted_labels)}")
        except ValueError:
            new_text_vectorized = vectorizer.transform([text]).toarray()
            pred = model.predict(new_text_vectorized)
            predicted_labels = mlb.inverse_transform(pred)[0]
            print(f"{name}: {', '.join(predicted_labels)}")

In [91]:
# Предсказание для нового текста
js_text = "Теперь о React и Vite. React — это мощная библиотека для построения пользовательских интерфейсов, основанная на компонентах. Она позволяет создавать переиспользуемые блоки, управлять состоянием и эффективно обновлять UI благодаря виртуальному DOM. Однако начальная настройка и сборка могут быть медленными. Vite, в свою очередь, — это инструмент сборки нового поколения, предлагающий молниеносную скорость разработки. Он использует ES-модули браузера для быстрого старта dev-сервера и оптимизирует сборку с помощью Rollup. React отлично подходит для сложных, компонентно-ориентированных приложений, тогда как Vite ускоряет процесс разработки, и их часто сочетают, чтобы получить лучшее от обоих миров!"
rust_text = "Rust — быстрый, безопасный язык программирования с фокусом на безопасность памяти. Actix — мощный фреймворк для Rust, идеален для создания быстрых, асинхронных веб-приложений. Вместе они обеспечивают высокую производительность, надежность и масштабируемость."
rust_linux = "Rust интегрируется с Linux для разработки безопасных и производительных компонентов ядра. Система владения и заимствования Rust предотвращает ошибки памяти, повышая надежность кода. Проект Rust-for-Linux позволяет писать драйверы, модули и подсистемы ядра, используя компилятор rustc для проверки ошибок на этапе сборки. Инструменты, такие как Cargo, управляют зависимостями, а bindgen генерирует привязки к C-коду для взаимодействия с существующей базой Linux. Библиотеки core и alloc адаптированы для работы в ядре без стандартной библиотеки std. Поддерживаются архитектуры, включая x86 и ARM, что расширяет возможности разработки. Примеры включают драйверы NVMe и USB, демонстрируя практическую применимость. Сообщество активно сотрудничает через GitHub и списки рассылки, обсуждая интеграцию и оптимизацию. Rust обеспечивает строгую типизацию и контроль конкурентности, минимизируя уязвимости. Это делает язык подходящим для критически важных систем, таких как Linux, где стабильность и безопасность приоритетны. Разработчики используют Rustup для управления версиями и QEMU для тестирования модулей. Интеграция упрощает создание масштабируемых и эффективных решений, снижая риск ошибок. Такой подход укрепляет экосистему Linux, открывая новые перспективы для системного программирования."
texts = [js_text, rust_text, rust_linux]

In [92]:
for txt in texts:
    print(txt)
    print("\nРезультаты тегирования:")
    extract_tags(txt)
    print("="*50)

Теперь о React и Vite. React — это мощная библиотека для построения пользовательских интерфейсов, основанная на компонентах. Она позволяет создавать переиспользуемые блоки, управлять состоянием и эффективно обновлять UI благодаря виртуальному DOM. Однако начальная настройка и сборка могут быть медленными. Vite, в свою очередь, — это инструмент сборки нового поколения, предлагающий молниеносную скорость разработки. Он использует ES-модули браузера для быстрого старта dev-сервера и оптимизирует сборку с помощью Rollup. React отлично подходит для сложных, компонентно-ориентированных приложений, тогда как Vite ускоряет процесс разработки, и их часто сочетают, чтобы получить лучшее от обоих миров!

Результаты тегирования:
TfidfVectorizer + LogisticRegression: javascript
TfidfVectorizer + SVC: javascript
CountVectorizer + LogisticRegression: javascript
CountVectorizer + SVC: 
Rust — быстрый, безопасный язык программирования с фокусом на безопасность памяти. Actix — мощный фреймворк для Rust,